In [1]:
import os
import glob
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from pyspark.sql.functions import col, month, year

import model_inference

def log1p_transform(x):
    return np.log1p(x)


In [2]:
# Build a .py script that takes a snapshot date, loads a model artefact and make an inference and save to datamart

## set up pyspark session

In [3]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/01 11:04:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/01 11:04:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/01 11:04:03 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## set up config

In [4]:
snapshot_date_str = "2008-03-01"
model_name = "diabetes_XGBoostmodel_2009_01_01.pkl"


In [5]:
config = {}
config["snapshot_date_str"] = snapshot_date_str
config["snapshot_date"] = datetime.strptime(config["snapshot_date_str"], "%Y-%m-%d")
config["model_name"] = model_name
config["model_bank_directory"] = "model_bank/"
config["model_artefact_filepath"] = config["model_bank_directory"] + config["model_name"]

pprint.pprint(config)

{'model_artefact_filepath': 'model_bank/diabetes_XGBoostmodel_2009_01_01.pkl',
 'model_bank_directory': 'model_bank/',
 'model_name': 'diabetes_XGBoostmodel_2009_01_01.pkl',
 'snapshot_date': datetime.datetime(2008, 3, 1, 0, 0),
 'snapshot_date_str': '2008-03-01'}


## load model artefact from model bank

In [6]:
# Load the model from the pickle file
with open(config["model_artefact_filepath"], 'rb') as file:
    model_artefact = pickle.load(file)

print("Model loaded successfully! " + config["model_artefact_filepath"])

Model loaded successfully! model_bank/diabetes_XGBoostmodel_2009_01_01.pkl


## load feature store

In [7]:
feature_location = "datamart/gold/feature_store/"

files_list = [feature_location + os.path.basename(f) for f in glob.glob(os.path.join(feature_location, '*'))]
feature_df = spark.read.option("header", "true").parquet(*files_list)
print("Feature DF row_count:", feature_df.count())

snapshot_date = datetime.strptime(config["snapshot_date_str"], "%Y-%m-%d")
target_year = snapshot_date.year
target_month = snapshot_date.month

# extract feature store
features_sdf = feature_df.filter((year(col('snapshot_date')) == target_year) & (month(col('snapshot_date')) == target_month))
print("extracted features_sdf", features_sdf.count(), config["snapshot_date"])

features_pdf = features_sdf.toPandas()
features_pdf

Feature DF row_count: 101766


extracted features_sdf 837 2008-03-01 00:00:00


,encounter_id,race_AfricanAmerican,race_Caucasian,race_Asian,race_Hispanic,is_female,age_midpoint,admission_severity_score,admission_source_risk_score,poor_glucose_control,metformin_ord,insulin_ord,diabetesMed,severity_x_visits,medication_density,diag_1_ord,diag_2_ord,diag_3_ord,snapshot_date
0,20839404,0,1,0,0,0,55,1,1,0,0,0,1,0,0.051923,2,0,0,2008-03-01
1,20847018,0,1,0,0,0,65,0,0,0,0,1,1,0,0.017857,10,0,1,2008-03-02
2,20852784,0,1,0,0,1,85,3,2,0,1,1,1,0,0.146774,10,10,0,2008-03-03
3,20853186,1,0,0,0,1,95,3,2,0,0,0,0,2,0.041958,0,1,0,2008-03-04
4,20855550,1,0,0,0,1,45,2,3,0,1,2,1,0,0.016717,0,7,0,2008-03-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,399665822,0,1,0,0,0,65,3,2,0,1,-1,1,3,0.028952,1,1,0,2008-03-27
833,399669368,0,1,0,0,0,25,3,2,0,0,0,1,4,0.022642,10,10,1,2008-03-28
834,399676502,0,1,0,0,1,75,1,1,0,1,0,1,0,0.043269,7,10,7,2008-03-29
835,399687794,0,0,1,0,1,75,3,2,0,0,1,1,4,0.036096,10,10,3,2008-03-30


## preprocess data for modeling

In [8]:
# prepare X_inference
feature_cols = [c for c in features_pdf.columns if c not in ["encounter_id", "snapshot_date", "label"]]

X_inference = features_pdf[feature_cols]

# apply transformer - standard scaler
transformer_stdscaler = model_artefact["preprocessing_transformers"]["stdscaler"]
X_inference = transformer_stdscaler.transform(X_inference)

print('X_inference', X_inference.shape[0])
X_inference

X_inference 837


array([[-0.69187694, -1.08373041, -0.8843236 , ...,  2.        ,
         0.        ,  0.        ],
       [-0.06305577, -2.03407603, -2.30418239, ..., 10.        ,
         0.        ,  1.        ],
       [ 1.19458655,  0.81696083,  0.53553518, ..., 10.        ,
        10.        ,  0.        ],
       ...,
       [ 0.56576539, -1.08373041, -0.8843236 , ...,  7.        ,
        10.        ,  7.        ],
       [ 0.56576539,  0.81696083,  0.53553518, ..., 10.        ,
        10.        ,  3.        ],
       [-0.06305577, -0.13338479,  0.53553518, ...,  1.        ,
         1.        ,  1.        ]], shape=(837, 17))

## model prediction inference

In [9]:
# load model
model = model_artefact["model"]

# predict model
y_inference = model.predict_proba(X_inference)[:, 1]

# prepare output
y_inference_pdf = features_pdf[["encounter_id", "snapshot_date"]].copy()
y_inference_pdf["model_name"] = config["model_name"]
y_inference_pdf["model_predictions"] = y_inference
y_inference_pdf

,encounter_id,snapshot_date,model_name,model_predictions
0,20839404,2008-03-01,diabetes_XGBoostmodel_2009_01_01.pkl,0.415957
1,20847018,2008-03-02,diabetes_XGBoostmodel_2009_01_01.pkl,0.357823
2,20852784,2008-03-03,diabetes_XGBoostmodel_2009_01_01.pkl,0.433348
3,20853186,2008-03-04,diabetes_XGBoostmodel_2009_01_01.pkl,0.509043
4,20855550,2008-03-05,diabetes_XGBoostmodel_2009_01_01.pkl,0.413823
...,...,...,...,...
832,399665822,2008-03-27,diabetes_XGBoostmodel_2009_01_01.pkl,0.551804
833,399669368,2008-03-28,diabetes_XGBoostmodel_2009_01_01.pkl,0.632385
834,399676502,2008-03-29,diabetes_XGBoostmodel_2009_01_01.pkl,0.451897
835,399687794,2008-03-30,diabetes_XGBoostmodel_2009_01_01.pkl,0.606221


## save model inference to datamart gold table

In [10]:
# create bronze datalake
gold_directory = f"datamart/gold/model_predictions/{config["model_name"][:-4]}/"
print(gold_directory)

if not os.path.exists(gold_directory):
    os.makedirs(gold_directory)

# save gold table - IRL connect to database to write
partition_name = config["model_name"][:-4] + "_predictions_" + snapshot_date_str.replace('-','_') + '.parquet'
filepath = gold_directory + partition_name
spark.createDataFrame(y_inference_pdf).write.mode("overwrite").parquet(filepath)
# df.toPandas().to_parquet(filepath,
#           compression='gzip')
print('saved to:', filepath)

datamart/gold/model_predictions/diabetes_XGBoostmodel_2009_01_01/


saved to: datamart/gold/model_predictions/diabetes_XGBoostmodel_2009_01_01/diabetes_XGBoostmodel_2009_01_01_predictions_2008_03_01.parquet


In [11]:
folder_path = f"datamart/gold/model_predictions/{config["model_name"][:-4]}/"
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
df = spark.read.option("header", "true").parquet(*files_list)
print("row_count:",df.count())

df.show()

row_count: 2457
+------------+-------------+--------------------+-------------------+
|encounter_id|snapshot_date|          model_name|  model_predictions|
+------------+-------------+--------------------+-------------------+
|   303790070|   2008-03-16|diabetes_XGBoostm...| 0.4071444571018219|
|   303795536|   2008-03-17|diabetes_XGBoostm...| 0.3861141502857208|
|   303801770|   2008-03-18|diabetes_XGBoostm...|0.30993562936782837|
|   303812330|   2008-03-19|diabetes_XGBoostm...|0.44413599371910095|
|   303816686|   2008-03-20|diabetes_XGBoostm...| 0.6213921904563904|
|   303819752|   2008-03-21|diabetes_XGBoostm...| 0.3772021532058716|
|   304230950|   2008-03-22|diabetes_XGBoostm...| 0.3523402214050293|
|   304242422|   2008-03-23|diabetes_XGBoostm...|  0.561093270778656|
|   304242458|   2008-03-24|diabetes_XGBoostm...| 0.3192260265350342|
|   304242692|   2008-03-25|diabetes_XGBoostm...| 0.5223513245582581|
|   304245878|   2008-03-26|diabetes_XGBoostm...|0.33316463232040405|
|   